# Create a Diverse Sample of Potentially Misogynistic Tweets

Rather than sampling randomly from a set of tweets that contain potentially misogynistic tokens, we build a K-means clustering implementation around these tweets (in bag-of-words vector space) and take the k cluster centers as our samples. 

This method allows us to create a diverse set of tweets (i.e., with little duplication in the tweet content), so our labelers will be exposed to a diverse set of content. This process reduces the redundancy in labeling, which is important given that we pay for people to label these tweets.

In [1]:
%matplotlib inline

In [2]:
import os
import json
import numpy as np
import matplotlib.pyplot as plt

## Read Our Random Sample of Misogynistic Tweets

These tweets were sampled from 2015 and contain a potentially misogynistic token.

In [3]:
tweets = sc.textFile("/home/ubuntu/remote/misogyny/misogyny.json.gz")

## Remove Sensisitve Tweets

"Possibly Sensitive" tweets often contain adult material, which we want to exclude. Pornography might mess up our labeling, as it isn't necessarily meant to be abusive but will likely use strongly vulgar keywords.

In [4]:
nonsens_twts = tweets.\
    filter(lambda tweet: "possibly_sensitive" not in json.loads(tweet)).\
    cache()
    
nonsens_twts_count = nonsens_twts.count()

In [5]:
print(nonsens_twts_count)

438774


# Pull tweet text and user

In [6]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer

enStops = stopwords.words("english")

In [7]:
import string

In [8]:
# Gotta pull in a bunch of packages for this
import sklearn.cluster
import sklearn.feature_extraction 
import sklearn.feature_extraction.text
import sklearn.metrics
import sklearn.preprocessing

from gensim import matutils
from gensim.corpora import Dictionary
from gensim.models import ldamulticore
from gensim.models import TfidfModel
from gensim.models.phrases import Phrases
from gensim.models.coherencemodel import CoherenceModel

from scipy import interpolate

import string

In [9]:
# Build a model from the data

# But first, read in stopwrods
enStop = stopwords.words('english')
esStop = stopwords.words('spanish')

# Skip stop words, retweet signs, @ symbols, and URL headers
stopList = enStop +\
    esStop +\
    ["http", "https", "rt", "@", ":", "co", "amp", "&amp;", "...", "\n", "\r"]
stopList.extend(string.punctuation)

vectorizer = sklearn.feature_extraction.text.CountVectorizer(strip_accents='unicode', 
                                                             tokenizer=TweetTokenizer(preserve_case=False).tokenize,
                                                             stop_words=stopList)

## Remove Retweets

Again, we prioritize limiting redundancy, so we delete retweets.

In [10]:
tweet_text = nonsens_twts.\
    map(lambda tweet_json: json.loads(tweet_json)).\
    filter(lambda tweet: "retweeted_status" not in tweet).\
    map(lambda tweet: tweet["text"]).\
    collect()

In [11]:
print("Non-sensitive, non-retweets:", len(tweet_text))

Non-sensitive, non-retweets: 261508


## Extract our Vocabulary

We will use the following vocabulary to build our bag of words vectorization model.

In [12]:
# all tweet text convenience variable
allTweetText = tweet_text

# Analyzer
analyze = vectorizer.build_analyzer() 
analyzed_text = [analyze(x) for x in allTweetText]

# Make bigrams from the text
bigram = Phrases(analyzed_text, min_count=20)
analyzed_text = [bigram[x] for x in analyzed_text]

dictionary = Dictionary(analyzed_text)



# Filter out words that occur too frequently or too rarely.
# Disregarding stop words, this dataset has a very high number of low frequency words.
max_freq = 0.9
min_count = 5
dictionary.filter_extremes(no_below=min_count, no_above=max_freq)

# This sort of "initializes" dictionary.id2token.
_ = dictionary[0]

# Create a map for vectorizer IDs to words
id2WordDict = dictionary.id2token
word2IdDict = dict(map(lambda x: (x[1], x[0]), id2WordDict.items()))

/home/ubuntu/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:486: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [15]:
# Create a bag of words
paired_corpus = list(
    filter(lambda x: len(x[1]) > 3, 
           [(idx, dictionary.doc2bow(text)) for idx, text in enumerate(analyzed_text)]))

corpus = [x[1] for x in paired_corpus]

# # Train TFIDF model
# tfidf_model = TfidfModel(corpus)

# # Built TFIDF-transformed corpus
# tfidf_corpus = [tfidf_model[text] for text in corpus]
tfidf_corpus = corpus

In [16]:
print("Dictionary Word Count:", len(id2WordDict))

Dictionary Word Count: 17846


## Use PySpark to Run k-Means

In [17]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.linalg import SparseVector


# Import data types
from pyspark.sql import Row
from pyspark.sql.types import StructType, ArrayType, FloatType

from pyspark.ml.clustering import KMeans

In [18]:
sparse_vectors = [Row(id=num, features=SparseVector(len(id2WordDict), x)) for num,x in enumerate(tfidf_corpus)]

In [19]:
df = spark.createDataFrame(sparse_vectors).cache()

## 1000-Element Clustering

In [ ]:
kmeans = KMeans(k=1000, seed=1)
model = kmeans.fit(df)
centers = model.clusterCenters()

In [ ]:
c_sizes = model.summary.clusterSizes
c_sizes

## Find the Exemplary Tweets for Each Cluster

That is, we want the tweet closest to the cluster center in vector space. This closest tweet is our exemplar tweet that we will use for labeling.

In [ ]:
exemplars = []

for index, center in enumerate(centers):
    top_k = min(5, c_sizes[index])
    v1_center = Vectors.dense(center)
    v1_dists = df.rdd.map(
        lambda row: (row.id, Vectors.squared_distance(row.features, v1_center))
    ).takeOrdered(top_k, key=lambda x: x[1])

    print("Cluster:", index, "Size:", c_sizes[index])
    first_flag = True
    for closest in v1_dists:
    
#         print("\tDistance to cluster center:", closest)
        if first_flag:
            exemplars.append(closest)
            first_flag = False
        
        (true_index, _) = paired_corpus[closest[0]]

        print("\t%s" % tweet_text[true_index])

Cluster: 252 Size: 817
	Them bitch ass talking shit
	LNAO this bitch always talking some shit.
	You're talking shit, bitch!
	@dashful0ranges keep talking shit bitch
	keep talking shit bitch.
Cluster: 253 Size: 356
	It's not uncommon for a hoe to get recognition
	@_therealbritnay it's *mad hoe! 😂
	@hayleymarie40 it's ok to be a side hoe
	it's the weekend stop being a hoe
	@GOODGlRLFAITH bc it's Christmas hoe
Cluster: 254 Size: 225
	you are a bad bitch and damn I like that
	Like , Damn you Stupid Bitch .
	Like damn bitch, mind your business !
	@elyssalovee orrrr he was like damn this bitch uglyyyyyyy
	When you catch a glimpse of yourself and you just like, "damn bitch"


In [36]:
for x in exemplars:
    (true_index, _) = paired_corpus[x[0]]

    print(tweet_text[true_index])

Bitch please. Stay in your lane..
like if you got a bitch say that
just doing bitch ass shit smh
@alexissssrusssh bitch take this off 😒😒😒😒😒😒😒😒😒😒
@ENVYShann stop being a hoe 😂😂🙄
@TasteMy_Smileey bitch no fuck that. I'm bruising yo shit
You ain't never have a bitch understand you
BITCH BETTER HAVE MY MONEY #MTVStars Rihanna ⚓ Nicki Minaj ⚓ Lady Gaga ⚓ One Direction ⚓
I can't have kids bitch get your mind right 😌
I hate it when ugly people say I need my beauty sleep. Bitch you need to hibernate.
took a nigga bitch 😂
See like a lil bitch
Hoe ass bitch nigga
shut ur bitch ass up .
4/300 BITCH VOTE!

#MTVStars Lady Gaga | Rihanna | Katy Perry | Ariana Grande | Beyonce | Iggy Azalea
@animooalien HOE DONT DO IT I NEED U IN MY LAIFEEEEE
Despite what everyone says, I don't think your mom is a "dirty pirate whore". dhe's actually quite clean for a  irate whore.
I'm such a bitch but I love it😂
When @BeffCrook 's mum answers the phone to you as 'Hollie the hoe' 👍🏼👍🏼👍🏼
I just really don't like this 

But I'm not gone argue with no bitch...
@Giavana_Raine @attianaa GIRL😂 bitch bye
"Rihanna - Bitch Better Have My Money (R3bab remix)"
Niggas really bitch made
You don't call your girlfriend a hoe. Where the real guys at
@etnow leave kinder alone you bitch
@Msrichards_2you bitch you was a tomboy 😩😭😭😭
@CSakua hoe ナグロビンうぇーい！#セフレ #sealds #男女 #nowar
Found a house party bitch!!!!!
@rubenbriones3 boy you a bitch all did was talk ain't nothing boy
yo bitch suckin' on my last name.
Guy: I WON THE LOTTERY! Girl: OMG! Guy: Pack your shit! Girl: What should I pack? Guy: Everything bitch you gotta go.
@BSfromPS motherfckrz like @AybekMazen are use to fckng there own sisterz you sick son of a bitch
I'm down to sprulge on my bitch kuz I don't wife no whores
She was a basic bitch till I piped her up.
@nsfwmason You're one lucky bitch
Can't take a hoe to serious
I see those colorful crayons on your face and bitch their not good
Boo bitch bye!👋🏽
@ThePrissAnn people like this fake bitch here.
if u work u

In [46]:
with open("full_diverse_sample.json", "w") as out_file:
    json.dump([tweet_text[y] for y in [paired_corpus[x[0]][0] for x in exemplars]], out_file)